In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

import time

In [2]:
def get_log_time():
    return datetime.now().strftime('%H:%M:%S %d/%m/%Y')

In [3]:
# ruta de carpera donde estan las llaves
print(get_log_time(), '> Estableciento conexion a base de datos...')
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud
print(get_log_time(), '> Base de datos conectado...!!!')

09:22:20 30/05/2023 > Estableciento conexion a base de datos...


09:22:27 30/05/2023 > Base de datos conectado...!!!


## Convertir a PDF

In [31]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/_3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx']

In [26]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


## SUBIR A DRIVE

In [27]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [28]:
contenido = os.listdir(ruta + r'//pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    cod = split[-1].split('.')
    dic_files[int(cod[0])] = file
print(len(dic_files), 'informes')
dic_files

2 informes


{108: '3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.pdf',
 109: '3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.pdf'}

In [29]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "EVAL_MANTENIMIENTO_PREVENTIVO"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta + r'//pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108.pdf with mimeType application/pdf
Created file 3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109.pdf with mimeType application/pdf
end


In [30]:
dic_urls

{108: 'https://drive.google.com/file/d/1KXXuq8EzJ8lAjZXQQKvOsoAcwvJArGHq/view?usp=drivesdk',
 109: 'https://drive.google.com/file/d/1iEaHiwr-RiCr4eFUJKi_cnZBUQbJVT3J/view?usp=drivesdk'}

In [31]:
for key in dic_urls:
    query = {'query': 'update dataset_315614 set link_reporte = \'{link}\' where id = {cod}'.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32920/sql', query)
    print(key)
print('end')

108
109
end


# GENERAR EN AUTOMATICO

In [99]:
def update_codigo_maquina(idd):
    query = {'query': 'update dataset_315614 set codigo_maquina = split_part(canhero, \' / \', 1) || \'-\' || codigo_flota where id={id_reg}'.format(id_reg = idd)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.post(project, query)

In [100]:
# generar reporte
def ECP_generar_reporte(reg):
    print(get_log_time(), '> (ECP) Inicio generar reporte:', reg)
    id_insp = reg['id']
    update_codigo_maquina(id_insp)
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if data['observaciones']==None:
        data['observaciones']=''
    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras-preventivo.docx')
    #descargar fotos y generar lista InlineImage
    print(get_log_time(), '> (ECP) Descargando imagenes...')
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(80)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(ruta + r'/informes/_' + file_name + '.docx')
    print(get_log_time(), '> (ECP) Reporte creado:', '_'+file_name)

In [101]:
# verifica si existen regiatros sin link de reporte
# si existe, reporna una lista 
def ECP_get_new_data():
    print(get_log_time() , '> (ECP) Buscando nuevos registros...')
    query_dia = {'query': 'select id, canhero from dataset_315614 where link_reporte = \'\' or link_reporte is null'}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select_dia = amigocloud.get(project, query_dia)
    data_dia = select_dia['data']
    print(get_log_time(), '> (ECP) Nuevos registros encontrados:', len(data_dia))
    return data_dia

In [102]:
# retorna una lista de los docuemntos ya generados pero que un no han sido revisados
def ECP_get_cod_files_exist():
    path = r'' + ruta + '/informes'
    contenido = os.listdir(path)
    k = []
    for i in contenido:
        items = i.split('_')
        cod_id = items[-1].split('.')[0]
        cod_insp = items[-4]
        k.append(cod_id + '_' + cod_insp)
    return k

In [103]:
# 
def ECP_eval_cosech_preventivo():
    # busca si existen registros sin reporte
    registros = ECP_get_new_data()
    if len(registros)>0: 
        for i in registros:
            # verifica si el reporte ya ha sido generado
            if str(i['id']) + '_' + 'ECP' in ECP_get_cod_files_exist():
                print('======= Reporte ya generado, falta revisar:', i)
                continue
            # en caso de no generado, generar
            ECP_generar_reporte(i)
        print(get_log_time(), '> (ECP) Se generaron todos los reportes.')
    else:
        print(get_log_time(), '> (ECP) No hay registros nuevos')

In [104]:
ECP_eval_cosech_preventivo()

10:20:06 30/05/2023 > (ECP) Buscando nuevos registros...
10:20:06 30/05/2023 > (ECP) Nuevos registros encontrados: 2
10:20:06 30/05/2023 > (ECP) Inicio generar reporte: {'id': 109, 'canhero': '3421 / ESPINOZA HURTADO ARMANDO'}
10:20:09 30/05/2023 > (ECP) Descargando imagenes...
10:20:24 30/05/2023 > (ECP) Reporte creado: _3421_ECP_25-05-2023_ESPINOZA HURTADO ARMANDO_109
10:20:24 30/05/2023 > (ECP) Inicio generar reporte: {'id': 108, 'canhero': '3400 / ESPINOZA AGUILAR JAIME OMAR '}
10:20:26 30/05/2023 > (ECP) Descargando imagenes...
10:20:37 30/05/2023 > (ECP) Reporte creado: _3400_ECP_24-05-2023_ESPINOZA AGUILAR JAIME OMAR _108
10:20:37 30/05/2023 > (ECP) Se generaron todos los reportes.


In [106]:
while True:
    
    time.sleep()

TypeError: sleep() takes exactly one argument (0 given)

In [120]:
get_files()

['108_ECP', '109_ECP']